# Postmodel Analysis

## Import relevant packages and settings

In [ ]:
import sys

sys.path.append("..")

%load_ext autoreload
%autoreload 2

In [ ]:
from config.project_constants import MODELING_CONFIG_FILE

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
sns.set_style("darkgrid")
sns.set_color_codes("muted")

In [ ]:
import altair as alt

alt.renderers.enable("default")

from src.utils.plot_util import (
    plot_scores_vs_actual_labels,
    compare_best_of_models,
    compare_inside_model_class,
)
from src.utils.sql_util import (
    get_db_conn,
    get_predictions_from_db,
    get_topk_models,
    get_best_model_from_model_class,
    get_metrics_from_db,
    get_saved_model_info_from_db,
    get_predictions_with_census_data,
    get_best_model_per_model_class,
    #get_feature_importance_from_db,
    get_predictions_with_feature_data,
    get_calibration_model,
    get_predictions_with_source_data,
)
from src.utils.pipeline_util import feature_importance
from src.utils.util import get_feature_groups_dict

In [ ]:
import yaml

with open("../" + MODELING_CONFIG_FILE) as f:
    modeling_config = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
db_conn = get_db_conn()

get_topk_models_wrapper = lambda db_conn, k=1: get_topk_models(db_conn=db_conn, k=k)
get_best_from_model_class_wrapper = (
    lambda db_conn, model_class, metric_of_interest: get_best_model_from_model_class(
        db_conn=db_conn, model_class=model_class, metric_of_interest=metric_of_interest
    )
)

In [ ]:
def threshold_scores(data, threshold=0.5):
    result_dict = {
        "y_true": data["y_true"].astype(int).values,
        "y_pred": (data["y_predicted"] >= threshold).astype(int).values,
    }
    return pd.DataFrame(result_dict)

In [ ]:
from datetime import date
import os

plots_folder_path = modeling_config["plots_folder_path"]
plots_folder_path_today = modeling_config["plots_folder_path"] + str(date.today())

# If plots_folder_path_today doesn't exist:
if not os.path.exists(plots_folder_path_today):
    os.makedirs(plots_folder_path_today)
    print(f"The new directory is created: {plots_folder_path_today}")

In [ ]:
plots_folder_path_today += "/"

## Analyse performance of the best model

In [ ]:
model_classes = [
    "AnswerRateAtCenter",
    "AdaBoostClassifier",
    "ExtraTreesClassifier",
    "ScaledLogisticRegression",
    "DecisionTreeClassifier",
    "RandomForestClassifier",
    "LogisticRegression",
]

results = get_best_model_per_model_class(
    db_conn=db_conn, model_class=model_classes, metric_of_interest="auc-roc"
)
display(results)

In [ ]:
top_model = get_topk_models_wrapper(db_conn, 1)
best_model_id = top_model.model_id[0]
best_model_name = top_model.model_class[0]
best_experiment_id = top_model.experiment_id[0]
display(top_model)

In [ ]:
print("These are the parameters that characterize the model:")
get_saved_model_info_from_db(
    db_conn=db_conn, model_id=best_model_id, info_to_get="parameters"
)[0]

In [ ]:
best_model_results = get_predictions_from_db(
    db_conn=db_conn, experiment_id=best_experiment_id, model_id=best_model_id
)
sorted_best_model_results = best_model_results.sort_values(
    by=["y_true", "y_predicted"],
    inplace=False,
    ascending=[False, False],
    na_position="last",
)
best_model_thresholded_results = threshold_scores(sorted_best_model_results)

print("The confusion matrix of the best model is:")
metrics.confusion_matrix(**best_model_thresholded_results)

In [ ]:
print(
    f"The base rate of the data is {best_model_results['y_true'].sum() / len(best_model_results['y_true'])}."
)

In [ ]:
print(
    f"Accuracy of the best model: {metrics.accuracy_score(**best_model_thresholded_results)}."
)
print(f"F1 of the best model: {metrics.f1_score(**best_model_thresholded_results)}.")
print(f"Worst AUC-ROC of the best model: {top_model['worst_value'][0]}.")

In [ ]:
best_model_fpr, best_model_tpr, thresholds = metrics.roc_curve(
    best_model_results["y_true"], best_model_results["y_predicted"]
)

## Analyse best baseline

In [ ]:
best_baseline = get_best_from_model_class_wrapper(
    db_conn,
    model_class=["AnswerRateAtCenter", "FeatureRanker"],
    metric_of_interest="auc-roc",
)
baseline_model_id = best_baseline.model_id[0]
baseline_experiment_id = best_baseline.experiment_id[0]
best_baseline

In [ ]:
print("These are the parameters that characterize the model:")
get_saved_model_info_from_db(
    db_conn=db_conn, model_id=best_model_id, info_to_get="parameters"
)[0]

In [ ]:
baseline_model_results = get_predictions_from_db(
    db_conn=db_conn, model_id=baseline_model_id, experiment_id=baseline_experiment_id
)
sorted_baseline_model_results = baseline_model_results.sort_values(
    by=["y_true", "y_predicted"],
    inplace=False,
    ascending=[False, False],
    na_position="last",
)
baseline_model_thresholded_results = threshold_scores(sorted_baseline_model_results)

print("The confusion matrix of the best baseline is:")
metrics.confusion_matrix(**baseline_model_thresholded_results)

In [ ]:
print(
    f"Accuracy of the best model: {metrics.accuracy_score(**baseline_model_thresholded_results)}."
)
print(
    f"F1 of the best model: {metrics.f1_score(**baseline_model_thresholded_results)}."
)
print(f"Best AUC-ROC of the best model: {best_baseline['worst_value'][0]}.")

In [ ]:
baseline_model_fpr, baseline_model_tpr, thresholds = metrics.roc_curve(
    baseline_model_results["y_true"], baseline_model_results["y_predicted"]
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
sns.lineplot(
    x=baseline_model_fpr,
    y=baseline_model_tpr,
    label=f"Feature ranker baseline, AUC={best_baseline['worst_value'][0]:.2f}",
    ax=ax,
    color="C0",
)
plt.title("AUC-ROC curve", fontsize=14)
plt.xlabel("FPR", fontsize=14)
plt.ylabel("TPR", fontsize=14)
plt.xlim(0.0, 1.01)
plt.ylim(0.0, 1.01)

fig.savefig(f"{plots_folder_path_today}auc_roc_best_baseline.png")

## Compare best model with best baseline

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
sns.lineplot(
    x=best_model_fpr,
    y=best_model_tpr,
    label=f"{best_model_name}, AUC={top_model['worst_value'][0]:.2f}",
    ax=ax,
    color="C1",
)
sns.lineplot(
    x=baseline_model_fpr,
    y=baseline_model_tpr,
    label=f"Baseline, AUC={best_baseline['worst_value'][0]:.2f}",
    ax=ax,
    color="C0",
)
plt.title("AUC-ROC curve", fontsize=14)
plt.xlabel("FPR", fontsize=14)
plt.ylabel("TPR", fontsize=14)
plt.xlim(0.0, 1.01)
plt.ylim(0.0, 1.01)

fig.savefig(f"{plots_folder_path_today}auc_roc_best_model_vs_best_baseline.png")

### Precision and Recall

In [ ]:
list_to_plot = {
    "Best model": sorted_best_model_results,
    "Best baseline": sorted_baseline_model_results,
}
fig, ax = plt.subplots(2, 2, figsize=(32, 10))
plt.subplots_adjust(left=0.1, bottom=0.2, right=0.4, top=0.8, wspace=0.4, hspace=0.4)

index = 0
for item, model_results in list_to_plot.items():
    (
        best_model_precision,
        best_model_recall,
        best_model_threshold,
    ) = metrics.precision_recall_curve(
        model_results["y_true"], model_results["y_predicted"]
    )

    # Precision-recall curve for best model
    sns.lineplot(
        x=best_model_threshold, y=best_model_precision[:-1], ax=ax[index][0], color="C0"
    )
    ax[index][0].set_xlabel("Threshold", fontsize=14)
    ax[index][0].set_ylabel("Precision", fontsize=14, color="C0")
    ax[index][0].set_ylim(0, 1)

    ax2 = ax[index][0].twinx()
    sns.lineplot(x=best_model_threshold, y=best_model_recall[:-1], ax=ax2, color="C1")
    ax2.set_ylabel("Recall", fontsize=14, color="C1", rotation=-90, labelpad=20)

    ax[index][0].set_title(item + " - Precision-Recall curve", fontsize=16)

    # Precision-recall curve for best model
    sns.lineplot(
        x=best_model_recall[:-1],
        y=best_model_precision[:-1],
        ax=ax[index][1],
        color="C0",
    )
    ax[index][1].set_xlabel("Recall", fontsize=14)
    ax[index][1].set_ylabel("Precision", fontsize=14)
    ax[index][1].set_title(item + " - Precision-Recall (best) curve", fontsize=16)
    ax[index][1].set_ylim(0, 1)

    index += 1

fig.savefig(
    f"{plots_folder_path_today}precision_recall_best_model_vs_best_baseline.png"
)

### Scores

In [ ]:
import plotly.io as pio

pio.renderers.default = "notebook"

In [ ]:
plot_scores_vs_actual_labels(
    best_model_results["y_true"],
    best_model_results["y_predicted"],
    show_plot=True,
    save_plot=True,
    title="Best model: distribution of the scores vs. the actual labels",
    plot_prefix_filename=f"{plots_folder_path_today}best_model_{best_model_id}_",
)

plot_scores_vs_actual_labels(
    baseline_model_results["y_true"],
    baseline_model_results["y_predicted"],
    show_plot=True,
    save_plot=True,
    title="Best baseline: distribution of the scores vs. the actual labels",
    plot_prefix_filename=f"{plots_folder_path_today}best_baseline_{baseline_model_id}_",
)

In [ ]:
compare_best_of_models(
    db_conn=db_conn,
    plot_folder_name=plots_folder_path_today,
    save_plot=True,
)

In [ ]:
compare_inside_model_class(
    db_conn=db_conn,
    model_class="ScaledLogisticRegression",
    plot_folder_name=plots_folder_path_today,
    save_plot=True,
    show_plot=True,
)

In [ ]:
compare_inside_model_class(
    db_conn=db_conn,
    model_class="RandomForestClassifier",
    plot_folder_name=plots_folder_path_today,
    save_plot=True,
    show_plot=True,
)

In [ ]:
compare_inside_model_class(
    db_conn=db_conn,
    model_class="DecisionTreeClassifier",
    plot_folder_name=plots_folder_path_today,
    save_plot=True,
    show_plot=True,
)

# Bias audit

In [ ]:
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
import aequitas.plot as ap

To get the complete list of available `variables_of_interest` you should first load the DataFrame `preds_and_census_data` and then run the following line of code:
> print(*preds_and_census_data.columns, sep="', '")

In [ ]:
variables_of_interest = [
    "population_size",
    "median_hh_income",
    "gini_index",
    "health_insurance_no_perc",
    "race_white_perc",
    "race_black_or_african_american_perc",
    "race_american_indian_and_alaska_native_perc",
    "race_asian_perc",
    "race_native_hawaiian_and_other_pacific_islander_perc",
    "race_hisp_latino_perc",
    "race_some_other_perc",
    "race_two_races_or_more_perc",
    "born_us_perc",
    "total_male_perc",
    "male_with_disability_perc",
    "female_with_disability_perc",
    "in_household_perc",
    "in_household_family_perc",
    "in_household_non_family_perc",
    "job_in_the_labor_force_employed_perc",
    "job_in_the_labor_force_unemployed_perc",
    "job_not_in_labor_force_perc",
]

### Bias audit at state level

In [ ]:
preds_and_census_data = get_predictions_with_census_data(
    db_conn=db_conn,
    experiment_id=top_model["experiment_id"][0],
    model_id=top_model["model_id"][0],
)

In [ ]:
for variable in variables_of_interest:
    preds_and_census_data[f"state_{variable}_categorized"] = pd.cut(
        preds_and_census_data[f"{variable}"],
        bins=preds_and_census_data[f"{variable}"]
        .quantile([0, 0.25, 0.5, 0.75, 1])
        .values,
        include_lowest=True,
        labels=["under_q25", "from_q25_to_q50", "from_q50_to_q75", "over_q75"],
    ).astype(str)
variables_of_interest_categorized = [
    "state_" + variable + "_categorized" for variable in variables_of_interest
]

In [ ]:
attributes_and_reference_groups = {"state": "Pennsylvania"}
for variable in variables_of_interest_categorized:
    attributes_and_reference_groups[variable] = "from_q50_to_q75"
attributes_to_audit = list(attributes_and_reference_groups.keys())

<div class="alert alert-warning">
    <b>Disclaimer!</b> The metric <i>AUC-ROC</i> is not implemented in the aequitas package. Hence, we compute it manually and we overwrite the metric <i>TPR</i> to take leverage of the ploting functionality of aequitas.
</div>

In [ ]:
df = preds_and_census_data[
    ["y_predicted", "y_true", "state"] + variables_of_interest_categorized
]
df = df.rename(columns={"y_predicted": "score", "y_true": "label_value"})

metrics_for_audit_bias = ["tpr"]
disparity_tolerance = 1.30

# Initialize Aequitas
g = Group()
b = Bias()

# get_crosstabs returns a dataframe of the group counts and group value bias metrics.
xtab, _ = g.get_crosstabs(df, attr_cols=attributes_to_audit)

for index, row in xtab.iterrows():
    attr_name = row["attribute_name"]
    attr_value = row["attribute_value"]

    df_attr_value = df[df[attr_name] == attr_value]
    xtab.loc[index, "tpr"] = metrics.roc_auc_score(
        y_true=df_attr_value["label_value"], y_score=df_attr_value["score"]
    )

bdf = b.get_disparity_predefined_groups(
    xtab, original_df=df, ref_groups_dict=attributes_and_reference_groups
)

We do a trick to save png charts since we have some problems with renders.

In [ ]:
for variable in attributes_and_reference_groups.keys():
    print(f"audit_bias_at_state_level_variable_{variable}.png")
    chart = ap.disparity(
        bdf, metrics_for_audit_bias, variable, fairness_threshold=disparity_tolerance
    )
    chart.display()
    chart.save(
        f"{plots_folder_path_today}audit_bias_at_state_level_variable_{variable}.html"
    )

### Bias audit at zcta level

In [ ]:
preds_and_census_data_zcta = get_predictions_with_census_data(
    db_conn=db_conn,
    experiment_id=top_model["experiment_id"][0],
    model_id=top_model["model_id"][0],
    geography="zcta",
)

In [ ]:
variables_of_interest_zcta = variables_of_interest

In [ ]:
skipped_variables = []
for variable in variables_of_interest_zcta:
    try:
        preds_and_census_data_zcta[f"zcta_{variable}_categorized"] = pd.cut(
            preds_and_census_data_zcta[f"{variable}"],
            bins=preds_and_census_data_zcta[f"{variable}"]
            .quantile([0, 0.25, 0.5, 0.75, 1])
            .values,
            include_lowest=True,
            labels=["under_q25", "from_q25_to_q50", "from_q50_to_q75", "over_q75"],
        ).astype(str)
    except:
        skipped_variables.append(variable)
variables_of_interest_categorized = [
    "zcta_" + variable + "_categorized"
    for variable in variables_of_interest
    if variable not in skipped_variables
]

In [ ]:
print(
    f"The following variables were skipped due to scarcity of data: {skipped_variables}."
)

In [ ]:
attributes_and_reference_groups = {
    "zcta": "94105"
}  # https://codigo-postal.co/en-us/usa/zip/94105/
for variable in variables_of_interest_categorized:
    attributes_and_reference_groups[variable] = "from_q50_to_q75"
attributes_to_audit = list(attributes_and_reference_groups.keys())

<div class="alert alert-warning">
    <b>Disclaimer!</b> The metric <i>AUC-ROC</i> is not implemented in the aequitas package. Hence, we compute it manually and we overwrite the metric <i>TPR</i> to take leverage of the ploting functionality of aequitas.
</div>

In [ ]:
import math

In [ ]:
df = preds_and_census_data_zcta[
    ["y_predicted", "y_true", "zcta"] + variables_of_interest_categorized
]
df = df.rename(columns={"y_predicted": "score", "y_true": "label_value"})

metrics_for_audit_bias = ["tpr"]
disparity_tolerance = 1.30

# Initialize Aequitas
g = Group()
b = Bias()

# get_crosstabs returns a dataframe of the group counts and group value bias metrics.
xtab, _ = g.get_crosstabs(df, attr_cols=attributes_to_audit)

for index, row in xtab.iterrows():
    attr_name = row["attribute_name"]
    attr_value = row["attribute_value"]

    df_attr_value = df[df[attr_name] == attr_value]
    try:
        xtab.loc[index, "tpr"] = metrics.roc_auc_score(
            y_true=df_attr_value["label_value"], y_score=df_attr_value["score"]
        )
    except:
        # If AUC-ROC cannot be computed, we remove the item from the list to avoid meaningless imputation.
        print(
            f"The combination of attr_name:{attr_name} and attr_value:{attr_value} has been deleted because auc-roc could not been computed."
        )
        xtab.drop(index)

    if math.isnan(xtab.loc[index, "tpr"]):
        xtab.drop(index)
        print(
            f"The combination of attr_name:{attr_name} and attr_value:{attr_value} has been deleted because auc-roc was nan."
        )

bdf = b.get_disparity_predefined_groups(
    xtab, original_df=df, ref_groups_dict=attributes_and_reference_groups
)

In [ ]:
bdf = bdf.dropna(subset=["tpr"])

In [ ]:
bdf.drop(bdf[bdf["tpr_disparity"] == 0].index, inplace=True)

In [ ]:
for variable in attributes_and_reference_groups.keys():
    print(f"audit_bias_model_{best_model_id}_at_zcta_level_variable_{variable}.png")
    chart = ap.disparity(
        bdf, metrics_for_audit_bias, variable, fairness_threshold=disparity_tolerance
    )
    chart.display()
    chart.save(
        f"{plots_folder_path_today}audit_bias_model_{best_model_id}_at_zcta_level_variable_{variable}.html"
    )

## ROC curves for race variables

In [ ]:
list_of_variables = [
    "race_white_perc",
    "race_black_or_african_american_perc",
    "race_american_indian_and_alaska_native_perc",
    "race_asian_perc",
    #"race_native_hawaiian_and_other_pacific_islander_perc",
    "race_hisp_latino_perc",
    "race_some_other_perc",
    "race_two_races_or_more_perc",
    "born_us_perc",
    "health_insurance_no_perc",
    "population_size",
]

In [ ]:
for variable in list_of_variables:
    df = preds_and_census_data_zcta[["y_true", "y_predicted", variable]]
    df[variable] = pd.cut(
        preds_and_census_data_zcta[f"{variable}"],
        bins=preds_and_census_data_zcta[f"{variable}"]
        .quantile([0, 0.25, 0.5, 0.75, 1])
        .values,
        include_lowest=True,
        labels=["under_q25", "from_q25_to_q50", "from_q50_to_q75", "over_q75"],
    ).astype(str)

    fig, ax = plt.subplots(1, 1, figsize=(8, 6))

    for index, v in enumerate(df[variable].unique()):
        df_v = df[df[variable] == v]
        df_v_model_fpr, df_v_model_tpr, thresholds = metrics.roc_curve(
            df_v["y_true"], df_v["y_predicted"]
        )

        sns.lineplot(
            x=df_v_model_fpr,
            y=df_v_model_tpr,
            label=f"{variable}_{v}",
            color=f"C{index}",
        )

    plt.title(f"AUC-ROC curve for {variable}", fontsize=14)
    plt.xlabel("FPR", fontsize=14)
    plt.ylabel("TPR", fontsize=14)
    plt.xlim(0.0, 1.01)
    plt.ylim(0.0, 1.01)

In [ ]:
predictions_with_source_data = get_predictions_with_source_data(
    db_conn=db_conn, model_id=best_model_id, experiment_id=best_experiment_id
)

In [ ]:
variable = "caller_state_full_name"
df = predictions_with_source_data[["y_true", "y_predicted", variable]]

auc_per_state = {}
for v in df[variable].unique():
    try:
        df_v = df[df[variable] == v]
        auc_per_state[v] = metrics.roc_auc_score(
            df_v["y_true"], df_v["y_predicted"]
        )
    except:
        continue

In [ ]:
dict(sorted(auc_per_state.items(), key=lambda x:x[1])))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

for index, v in enumerate(df[variable].unique()):
    try:
        df_v = df[df[variable] == v]
        df_v_model_fpr, df_v_model_tpr, thresholds = metrics.roc_curve(
            df_v["y_true"], df_v["y_predicted"]
        )
        
        if v in ["Nebraska", "District of Columbia"]:
            color = "red"
            label = True
        elif v in ["Colorado", "California"]:
            color = "orange"
            label = True
        elif v in ["Tennessee", "New Mexico"]:
            color = "green"
            label = True
        else:
            color = "gray"
            label = False

        if label:
            sns.lineplot(
                x=df_v_model_fpr,
                y=df_v_model_tpr,
                label=f"{v}",
                color=color,
            )
        else:
            sns.lineplot(
                x=df_v_model_fpr,
                y=df_v_model_tpr,
                color=color,
            )
    except:
        continue

plt.title(f"AUC-ROC curve for {variable}", fontsize=14)
plt.xlabel("FPR", fontsize=14)
plt.ylabel("TPR", fontsize=14)
plt.xlim(0.0, 1.01)
plt.ylim(0.0, 1.01)

In [ ]:
predictions_with_source_data["center_key"]

In [ ]:
#predictions_with_source_data = get_predictions_with_source_data(
#    db_conn=db_conn, model_id=best_model_id, experiment_id=best_experiment_id
#)
variable = "center_key"
df = predictions_with_source_data[["y_true", "y_predicted", variable]]

auc_per_center = {}
for v in df[variable].unique():
    try:
        df_v = df[df[variable] == v]
        auc_per_center[v] = metrics.roc_auc_score(
            df_v["y_true"], df_v["y_predicted"]
        )
    except:
        continue

In [ ]:
dict(sorted(auc_per_center.items(), key=lambda x:x[1]))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

for index, v in enumerate(df[variable].unique()):
    try:
        df_v = df[df[variable] == v]
        df_v_model_fpr, df_v_model_tpr, thresholds = metrics.roc_curve(
            df_v["y_true"], df_v["y_predicted"]
        )
        
        if v in ["PA724123", "GU000671"]:
            color = "red"
            label = True
        elif v in ["MI660000", "IL410000"]:
            color = "orange"
            label = True
        elif v in ["NY122000", "OR503000"]:
            color = "green"
            label = True
        else:
            color = "gray"
            label = False

        if label:
            sns.lineplot(
                x=df_v_model_fpr,
                y=df_v_model_tpr,
                label=f"{v}",
                color=color,
            )
        else:
            sns.lineplot(
                x=df_v_model_fpr,
                y=df_v_model_tpr,
                color=color,
            )
    except:
        continue

plt.title(f"AUC-ROC curve for {variable}", fontsize=14)
plt.xlabel("FPR", fontsize=14)
plt.ylabel("TPR", fontsize=14)
plt.xlim(0.0, 1.01)
plt.ylim(0.0, 1.01)

In [ ]:
predictions_with_source_data = get_predictions_with_source_data(
    db_conn=db_conn, model_id=best_model_id, experiment_id=best_experiment_id
)

In [ ]:
variable = "arrived_part_of_day"
df = predictions_with_source_data[["y_true", "y_predicted", variable]]
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

for index, v in enumerate(df[variable].unique()):
    try:
        df_v = df[df[variable] == v]
        df_v_model_fpr, df_v_model_tpr, thresholds = metrics.roc_curve(
            df_v["y_true"], df_v["y_predicted"]
        )
        
        sns.lineplot(
            x=df_v_model_fpr,
            y=df_v_model_tpr,
            label=f"{v}",
        )
    except:
        continue

plt.title(f"AUC-ROC curve for {variable}", fontsize=14)
plt.xlabel("FPR", fontsize=14)
plt.ylabel("TPR", fontsize=14)
plt.xlim(0.0, 1.01)
plt.ylim(0.0, 1.01)

## Feature Importance

In [ ]:
feature_importance_data = get_feature_importance_from_db(
    db_conn=db_conn, model_id=best_model_id
)

In [ ]:
feature_importance_data

In [ ]:
top_features = 20

f, ax = plt.subplots(figsize=(6, 15))
sns.barplot(
    x="feature_importance",
    y="feature_name",
    color="b",
    data=feature_importance_data[:top_features],
)
sns.despine(left=True, bottom=True)

f.savefig(
    f"{plots_folder_path_today}feature_importance_top{top_features}_model_{best_model_id}.png"
)

In [ ]:
feature_groups_dict = get_feature_groups_dict(modeling_config["feature_config"])

In [ ]:
feature_importance_data_index = feature_importance_data.set_index("feature_name")

# The following is a temporal fix tbecause the features table has not been updated with the latest feature names:
feature_importance_data_index = feature_importance_data_index.rename(
    index={"arrived_week_of_year_numerics": "arrived_datetime_est_week_of_year_numeric"}
)

feature_groups_dict_feature_importance = {}
for group, variables in feature_groups_dict.items():
    if group in ["datetime", "zero_param_alphabet_soup", "datetime_imputed"]:
        try:
            variables = sum(variables, [])
            variables.remove("arrived_datetime_local_week_of_year_numeric")
        except:
            pass

    try:
        feature_groups_dict_feature_importance[group] = max(
            feature_importance_data_index.loc[variables, "feature_importance"]
        )
    except:
        pass

In [ ]:
feature_groups_feature_importance = pd.DataFrame(
    list(
        zip(
            feature_groups_dict_feature_importance.keys(),
            feature_groups_dict_feature_importance.values(),
        )
    ),
    columns=["feature_group", "max_feature_importance"],
)

f, ax = plt.subplots(figsize=(8, 6))
sns.despine(left=True, bottom=True)

ax = sns.barplot(
    x="feature_group",
    y="max_feature_importance",
    color="b",
    data=feature_groups_feature_importance,
)
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=30)

f.savefig(
    f"{plots_folder_path_today}feature_group_max_importance_model_{best_model_id}.png"
)

## Calibration

In [ ]:
calibration_data = get_calibration_model(db_conn=db_conn, model_id=best_model_id)

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))

sns.scatterplot(data=calibration_data, x="avg_y_pred", y="avg_y_true")
plt.xlabel("Average predicted score", fontsize=14)
plt.ylabel("Average true label", fontsize=14)
_ = plt.title("Is the model calibrated?", fontsize=16)

f.savefig(f"{plots_folder_path_today}calibration_results_model_{best_model_id}.png")

In [ ]:
calibration_data = get_calibration_model(
    db_conn=db_conn, model_id=best_model_id, round_value=2
)

f, ax = plt.subplots(figsize=(8, 6))

sns.scatterplot(data=calibration_data, x="avg_y_pred", y="avg_y_true")
plt.xlabel("Average predicted score", fontsize=14)
plt.ylabel("Average true label", fontsize=14)
_ = plt.title("Is the model calibrated?", fontsize=16)

f.savefig(
    f"{plots_folder_path_today}calibration_results_precision2_model_{best_model_id}.png"
)

In [ ]:
calibration_data = get_calibration_model(
    db_conn=db_conn, model_id=best_model_id, round_value=3
)

f, ax = plt.subplots(figsize=(8, 6))

sns.scatterplot(data=calibration_data, x="avg_y_pred", y="avg_y_true")
plt.xlabel("Average predicted score", fontsize=14)
plt.ylabel("Average true label", fontsize=14)
_ = plt.title("Is the model calibrated?", fontsize=16)

f.savefig(
    f"{plots_folder_path_today}calibration_results_precision3_model_{best_model_id}.png"
)

In [ ]:
calibration_data = get_calibration_model(
    db_conn=db_conn, model_id=best_model_id, round_value=4
)

f, ax = plt.subplots(figsize=(8, 6))

sns.scatterplot(data=calibration_data, x="avg_y_pred", y="avg_y_true")
plt.xlabel("Average predicted score", fontsize=14)
plt.ylabel("Average true label", fontsize=14)
_ = plt.title("Is the model calibrated?", fontsize=16)

f.savefig(
    f"{plots_folder_path_today}calibration_results_precision4_model_{best_model_id}.png"
)

## Crosstabs

In [ ]:
from src.utils.sql_util import (
    get_predictions_with_feature_data,
)

In [ ]:
predictions_with_feature_data = get_predictions_with_feature_data(
    db_conn=db_conn, model_id=best_model_id, experiment_id=best_experiment_id
)

In [ ]:
predictions_with_feature_data.head()

In [ ]:
k = 0.5
df = predictions_with_feature_data
df["score"] = pd.cut(
    predictions_with_feature_data["y_predicted"], [0, k, 1], labels=[f"<{k}", f">={k}"]
)

In [ ]:
df = df.drop(
    columns=[
        "routing_attempts_id",
        "Unnamed: 0",
        "initiated_datetime_est",
        "initiated_datetime_est.1",
    ]
)

In [ ]:
import scipy.stats as stats

In [ ]:
crosstabs_data = pd.DataFrame(
    {
        f"<{k}": df[df["score"] == f"<{k}"].mean(axis=0),
        f">={k}": df[df["score"] == f">={k}"].mean(axis=0),
        f"n_<{k}": df[df["score"] == f"<{k}"].count(),
        f"n_>={k}": df[df["score"] == f">={k}"].count(),
    }
)

In [ ]:
ratio1 = crosstabs_data[f"<{k}"] / crosstabs_data[f">={k}"]
ratio2 = crosstabs_data[f">={k}"] / crosstabs_data[f"<{k}"]
crosstabs_data["max_ratio"] = np.where(ratio1 > ratio2, ratio1, ratio2)
crosstabs_data["max_ratio"] = crosstabs_data["max_ratio"].replace([np.inf, -np.inf], -1)

In [ ]:
crosstabs_data["max_ratio"].describe()

In [ ]:
crosstabs_data[f"flag_<{k}_between0and1"] = crosstabs_data[f"<{k}"] < 1
crosstabs_data[f"flag_>={k}_between0and1"] = crosstabs_data[f">={k}"] < 1
crosstabs_data["flag_between0and1"] = np.where(
    crosstabs_data[f"flag_>={k}_between0and1"],
    True,
    crosstabs_data[f"flag_<{k}_between0and1"],
)

In [ ]:
crosstabs_data["mannwhitneyu_pvalue"] = -1
for attr in crosstabs_data.index:
    crosstabs_data.loc[attr, "mannwhitneyu_pvalue"] = stats.mannwhitneyu(
        df.loc[df["score"] == f"<{k}", attr],
        df.loc[df["score"] == f">={k}", attr],
        alternative="two-sided",
    )[1]

In [ ]:
crosstabs_data = crosstabs_data.sort_values(
    by=["flag_between0and1", "max_ratio"], ascending=[True, False]
)
pd.set_option("display.max_rows", 650)
crosstabs_data

## Error analysis

In [ ]:
from src.utils.sql_util import get_predictions_with_source_data

In [ ]:
predictions_with_source_data = get_predictions_with_source_data(
    db_conn=db_conn, model_id=best_model_id, experiment_id=best_experiment_id
)

In [ ]:
predictions_with_source_data

In [ ]:
attributes_and_reference_groups = {
    "center_key": "PA215000",
    "arrived_part_of_day": "morning",
}
attributes_to_audit = list(attributes_and_reference_groups.keys())

<div class="alert alert-warning">
    <b>Disclaimer!</b> The metric <i>AUC-ROC</i> is not implemented in the aequitas package. Hence, we compute it manually and we overwrite the metric <i>TPR</i> to take leverage of the ploting functionality of aequitas.
</div>

In [ ]:
import math

In [ ]:
df = predictions_with_source_data[["y_predicted", "y_true"] + attributes_to_audit]
df = df.rename(columns={"y_predicted": "score", "y_true": "label_value"})

metrics_for_audit_bias = ["tpr"]
disparity_tolerance = 1.30

# Initialize Aequitas
g = Group()
b = Bias()

# get_crosstabs returns a dataframe of the group counts and group value bias metrics.
xtab, _ = g.get_crosstabs(df, attr_cols=attributes_to_audit)

for index, row in xtab.iterrows():
    attr_name = row["attribute_name"]
    attr_value = row["attribute_value"]

    df_attr_value = df[df[attr_name] == attr_value]
    try:
        xtab.loc[index, "tpr"] = metrics.roc_auc_score(
            y_true=df_attr_value["label_value"], y_score=df_attr_value["score"]
        )
    except:
        # If AUC-ROC cannot be computed, we remove the item from the list to avoid meaningless imputation.
        print(
            f"The combination of attr_name:{attr_name} and attr_value:{attr_value} has been deleted because auc-roc could not been computed."
        )
        xtab.drop(index)

    if math.isnan(xtab.loc[index, "tpr"]):
        xtab.drop(index)
        print(
            f"The combination of attr_name:{attr_name} and attr_value:{attr_value} has been deleted because auc-roc was nan."
        )

bdf = b.get_disparity_predefined_groups(
    xtab, original_df=df, ref_groups_dict=attributes_and_reference_groups
)

In [ ]:
bdf.drop(bdf[bdf["tpr_disparity"] == 0].index, inplace=True)

In [ ]:
for variable in attributes_and_reference_groups.keys():
    print(
        f"audit_bias_model_{best_model_id}_absolute_at_cohort_level_variable_{variable}.png"
    )
    chart = ap.absolute(
        bdf, metrics_for_audit_bias, variable, fairness_threshold=disparity_tolerance
    )
    chart.display()
    chart.save(
        f"{plots_folder_path_today}audit_bias_model_{best_model_id}_absolute_at_cohort_level_variable_{variable}.html"
    )